In [ ]:
from huggingface_hub import login
login()

In [ ]:
%%capture
!pip install -U datasets
!pip install -U trl
!pip install -U transformers accelerate
!pip install -U bitsandbytes
# # # # # # uv pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
# from dotenv import load_dotenv
import os
from peft import LoraConfig
from peft import TaskType
from trl import SFTConfig, SFTTrainer
from transformers import BitsAndBytesConfig
# load_dotenv()

from huggingface_hub import login
login()

print("torch version: ", torch.__version__)
print('is gpu enbled: ', torch.cuda.is_available())

torch version:  2.6.0+cu124
is gpu enbled:  False


In [ ]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [ ]:
# Load model and tokenizer
model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
base_model = AutoModelForCausalLM.from_pretrained(model_id,
                                                  quantization_config=bnb_config,
                                                  device_map='auto')
# meta-llama/Llama-3.2-3B-Instruct

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
# for name, weights in base_model.named_parameters():
#     print(name)
#     print(weights)
#     break

model.embed_tokens.weight
Parameter containing:
tensor([[ 1.1292e-02,  9.9487e-03,  1.4160e-02,  ..., -3.5706e-03,
         -1.9775e-02,  5.3711e-03],
        [ 1.3245e-02, -3.8385e-05,  2.2461e-02,  ..., -2.6550e-03,
          3.1738e-02, -1.0681e-03],
        [ 1.9775e-02,  2.0020e-02,  2.8687e-02,  ..., -3.5248e-03,
          3.1433e-03, -7.6294e-03],
        ...,
        [-3.0975e-03,  2.1057e-03,  4.8828e-03,  ..., -2.0905e-03,
         -1.2207e-03, -2.8992e-03],
        [-3.0975e-03,  2.1057e-03,  4.8828e-03,  ..., -2.0905e-03,
         -1.2207e-03, -2.8992e-03],
        [-3.0975e-03,  2.1057e-03,  4.8828e-03,  ..., -2.0905e-03,
         -1.2207e-03, -2.8992e-03]], device='cuda:0', dtype=torch.float16,
       requires_grad=True)


In [ ]:
# Load the dataset
from datasets import load_dataset
ds = load_dataset("Jofthomas/hermes-function-calling-thinking-V1")

README.md:   0%|          | 0.00/354 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.85M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3570 [00:00<?, ? examples/s]

In [ ]:
# ds['train'][12] # sample data

In [ ]:
sample_data = ds['train'][0]['conversations']

In [ ]:
# we won't use bos token in here , since it's will be added when we set SFTTrainer and data get tokenized, so we set new chat template there with bos token and no other difference
chat_template_for_preprocessing = "{% if  messages[0]['role']==system%}{{ raise_exception('System message is not supported in gemma, it would be good to merget the system prompt with first user message')}}{% endif %}{% for message in messages %}{{'<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<eos_turn><eos>\n'}}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"
chat_template = "{{ bos_token }}{% if  messages[0]['role']==system%}{{ raise_exception('System message is not supported in gemma, it would be good to merget the system prompt with first user message')}}{% endif %}{% for message in messages %}{{'<start_of_turn>' + message['role'] + '\n' + message['content'] | trim + '<eos_turn><eos>\n'}}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"


In [ ]:
tokenizer.chat_template = chat_template_for_preprocessing

In [ ]:
print(tokenizer.apply_chat_template(sample_data, tokenize=False))

<start_of_turn>system
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'Get the current stock price of a company', 'parameters': {'type': 'object', 'properties': {'company': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company']}}}, {'type': 'function', 'function': {'name': 'get_movie_details', 'description': 'Get details about a movie', 'parameters': {'type': 'object', 'properties': {'title': {'type': 'string', 'description': 'The title of the movie'}}, 'required': ['title']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Argumen

In [ ]:
ds = ds.rename_column('conversations', 'messages')

In [ ]:
# preprocess dataset {apply chat template}

def preprocess_dataset(row):
    messages = row['messages'] # Dict['str', List[Dict['str', 'str']]]

    # Check if there system message, if yes, merge the system prompt with first user input, since the gemma model does not have system prompt , it trained in this way
    if messages[0]['role'] == 'system':
        system_message = messages[0]['content']
        messages[1]['content'] = system_message + "Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thought}</think>\n\n" + messages[1]['content']
        # after merging, remove the system message from message
        messages.pop(0)

    return {'text': tokenizer.apply_chat_template(messages, tokenize=False)}


In [ ]:
ds = ds.map(preprocess_dataset, remove_columns=['messages'])

Map:   0%|          | 0/3570 [00:00<?, ? examples/s]

In [ ]:
ds = ds['train'].train_test_split(.1, seed=0)

In [ ]:
test_split = ds['test'].train_test_split(0.4, seed=0)

ds['validation'] = test_split['train']
ds['test'] = test_split['test']

In [ ]:
from dataclasses import dataclass
from enum import Enum

class ChatToolLMSpecialTokens(Enum):
    tools = '<tools>'
    eos_tools = '</tools>'
    think = '<think>'
    eos_think = '</think>'
    tool_response = '<tool_response>'
    eos_tool_response = '</tool_response>'
    pad_token = '<pad>'
    eos_token = '<eos>'

    @classmethod
    def special_token_list(clss):
        return [cls_token.value for cls_token in clss]

ChatToolLMSpecialTokens.special_token_list()


['<tools>',
 '</tools>',
 '<think>',
 '</think>',
 '<tool_response>',
 '</tool_response>',
 '<pad>',
 '<eos>']

In [ ]:
list(ChatToolLMSpecialTokens)

[<ChatToolLMSpecialTokens.tools: '<tools>'>,
 <ChatToolLMSpecialTokens.eos_tools: '</tools>'>,
 <ChatToolLMSpecialTokens.think: '<think>'>,
 <ChatToolLMSpecialTokens.eos_think: '</think>'>,
 <ChatToolLMSpecialTokens.tool_response: '<tool_response>'>,
 <ChatToolLMSpecialTokens.eos_tool_response: '</tool_response>'>,
 <ChatToolLMSpecialTokens.pad_token: '<pad>'>,
 <ChatToolLMSpecialTokens.eos_token: '<eos>'>]

In [ ]:
ChatToolLMSpecialTokens.pad_token.value

'<pad>'

In [ ]:
# Load tokenizer again so we can add special tokens easly
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          pad_token=ChatToolLMSpecialTokens.pad_token.value,
                                          eos_token=ChatToolLMSpecialTokens.eos_token.value,
                                          additional_special_tokens=ChatToolLMSpecialTokens.special_token_list())

In [ ]:
# set new chat template with bos token for sft auto data processing
tokenizer.chat_template = chat_template

In [ ]:
peft_config = LoraConfig(r=64,
                         lora_alpha=128,
                         target_modules=['q_proj', 'v_proj', 'o_proj', 'embed_tokens', 'k_proj', 'lm_head', 'up_proj'], # 'gate_proj', 'up_proj', 'down_proj'],
                         task_type=TaskType.CAUSAL_LM,
                         lora_dropout=0.01)

In [ ]:
len(tokenizer) # Total number of tokens

256006

In [ ]:
# since we added some special tokenzer , we need to add some random embedding for it
base_model.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128264, 3072)

In [ ]:
from peft import get_peft_model, prepare_model_for_kbit_training

base_model.gradient_checkpointing_enable()

peft_model = prepare_model_for_kbit_training(base_model)
peft_model = get_peft_model(peft_model, peft_config=peft_config)

peft_model.config.use_cache = False

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:574: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens', 'lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [ ]:
peft_model.print_trainable_parameters()

trainable params: 53,511,168 || all params: 3,266,285,568 || trainable%: 1.6383


In [ ]:
training_arugment = SFTConfig(output_dir='gemma_function_calling_and_thinking',
                              per_device_train_batch_size=1,
                              per_device_eval_batch_size=1,
                              gradient_accumulation_steps=4,
                              optim="paged_adamw_8bit",
                              logging_first_step=True,
                              logging_dir='runs',
                              learning_rate=1e-4,
                              max_grad_norm=1.0,
                              num_train_epochs=2,
                              warmup_ratio=0.1,
                              lr_scheduler_type='cosine',
                              eval_strategy='steps',
                              save_strategy='steps',
                              report_to='tensorboard',
                              gradient_checkpointing=True,
                              # gradient_checkpointing_kwargs={"use_reentrant": False},
                              packing=False,
                              save_steps=100,
                              eval_steps=10,
                              logging_steps=10,
                              fp16=True,
                              max_seq_length=1500)

In [ ]:
trainer =  SFTTrainer(model=peft_model,
                      processing_class=tokenizer,
                      peft_config=peft_config,
                      train_dataset=ds['train'],
                      eval_dataset=ds['validation'].select(range(50)),
                      args=training_arugment)

Adding EOS to train dataset:   0%|          | 0/3213 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/3213 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/3213 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
peft_model.device

device(type='cuda', index=0)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
10,1.762600,1.776343
20,1.675400,1.612458
30,1.519800,1.401565
40,1.290000,1.116007
50,1.017100,0.825638
60,0.850900,0.711902
70,0.712600,0.659577
80,0.653800,0.596729
90,0.561100,0.485253
100,0.464000,0.410044


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


KeyboardInterrupt: 

## Infernce

In [ ]:
dataloader = trainer.get_train_dataloader()

In [ ]:
for i in dataloader:
    break

In [ ]:
tokenizer.decode(tokenizer.all_special_ids)

In [ ]:
print(tokenizer.batch_decode(i['input_ids'])[0])

In [ ]:
inp = tokenizer.batch_decode(i['input_ids'])[0]
labels = tokenizer.batch_decode(i['labels'])[0]
inp==labels

In [ ]:
# len(i['labels'][0])

In [ ]:
# print(labels)

In [ ]:
# tokenizer.decode(i['input_ids'][0])

In [ ]:
##inference

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

peft_model_id = "Abdulvajid/Llama_Qlora_Reasoning_ToolCalling_Finetuned_4Bit"

model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, torch_dtype=torch.float16, device_map='cuda', load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
model.eval();

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:574: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['model.embed_tokens', 'lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


In [ ]:
## chat template

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

In [ ]:
input = """<|begin_of_text|><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'search_restaurants', 'description': 'Search for restaurants in a specific location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to search for restaurants'}, 'cuisine': {'type': 'string', 'description': 'The cuisine type to filter the restaurants'}, 'price_range': {'type': 'integer', 'description': 'The price range of the restaurants (1-4)'}}, 'required': ['location']}}}, {'type': 'function', 'function': {'name': 'generate_random_password', 'description': 'Generate a random password', 'parameters': {'type': 'object', 'properties': {'length': {'type': 'integer', 'description': 'The length of the password'}}, 'required': ['length']}}}] </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call>Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thought}</think>

Im in kerala in india, currently i am in city called malappuram, can you find some restuarant here<eos_turn><eos>
<start_of_turn>model"""

In [ ]:
inp = tokenizer.encode(input, return_tensors='pt',add_special_tokens=False).to('cuda')

In [ ]:
print(tokenizer.decode(inp[0]))

<|begin_of_text|><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name':'search_restaurants', 'description': 'Search for restaurants in a specific location', 'parameters': {'type': 'object', 'properties': {'location': {'type':'string', 'description': 'The location to search for restaurants'}, 'cuisine': {'type':'string', 'description': 'The cuisine type to filter the restaurants'}, 'price_range': {'type': 'integer', 'description': 'The price range of the restaurants (1-4)'}},'required': ['location']}}}, {'type': 'function', 'function': {'name': 'generate_random_password', 'description': 'Generate a random password', 'parameters': {'type': 'object', 'properties': {'length': {'type': 'integer', 'description

In [ ]:
out = model.generate(inp, max_new_tokens=200)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(tokenizer.decode(out[0]))

<|begin_of_text|><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name':'search_restaurants', 'description': 'Search for restaurants in a specific location', 'parameters': {'type': 'object', 'properties': {'location': {'type':'string', 'description': 'The location to search for restaurants'}, 'cuisine': {'type':'string', 'description': 'The cuisine type to filter the restaurants'}, 'price_range': {'type': 'integer', 'description': 'The price range of the restaurants (1-4)'}},'required': ['location']}}}, {'type': 'function', 'function': {'name': 'generate_random_password', 'description': 'Generate a random password', 'parameters': {'type': 'object', 'properties': {'length': {'type': 'integer', 'description

In [ ]:
print(tokenizer.decode(ds['train'][0]['messages']))

In [ ]:
trainer.push_to_hub("Abdulvajid/Llama_Qlora_Reasoning&ToolCalling_Finetuned_4Bit")
# tokenizer.push_to_hub("Abdulvajid/gemma2b_it_toolcalling_finetuned")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:222: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/3.37G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.69k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Abdulvajid/gemma_function_calling_and_thinking/commit/540478e4ce54ed05e0965f3d5c33842aa98b71f7', commit_message='Abdulvajid/Llama_Qlora_Reasoning&ToolCalling_Finetuned_4Bit', commit_description='', oid='540478e4ce54ed05e0965f3d5c33842aa98b71f7', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Abdulvajid/gemma_function_calling_and_thinking', endpoint='https://huggingface.co', repo_type='model', repo_id='Abdulvajid/gemma_function_calling_and_thinking'), pr_revision=None, pr_num=None)

In [1]:
from transformers import AutoTokenizer
peft_model_id = "Abdulvajid/Llama_Qlora_Reasoning_ToolCalling_Finetuned_4Bit"
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [9]:
messages = [
    {"role": "user", "content": "I'm in Malappuram, find a restaurant."}
]

In [80]:
template = '''{{ bos_token }}{% if messages[0].role == 'system' %}
{{ raise_exception('System message is not supported in gemma; merge the system prompt with the first user message') }}
{% endif %}
{% if not tools %}
{{ raise_exception('You should provide tools; this model was only trained for tool calling with reasoning') }}
{% endif %}
{% set first_message = True %}
{% for message in messages %}
  {% if first_message and tools %}
{% raw -%}
<start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> {% endraw %}{{ tools }}{% raw -%} </tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']}For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:
<tool_call>
{tool_call}
</tool_call>Also, before making a call to a function take the time to plan the function to take. Make that thinking process between <think>{your thought}</think>

{% endraw %}{{ message.content }}{% raw -%}
<eos_turn><eos>
<start_of_turn>model
{% endraw -%}
    {% set first_message = False %}
  {% endif %}
{% endfor %}
'''

In [81]:
tokenizer.chat_template = template

In [82]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_restaurants",
            "description": "Search for restaurants in a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location to search for restaurants"
                    },
                    "cuisine": {
                        "type": "string",
                        "description": "The cuisine type to filter the restaurants"
                    },
                    "price_range": {
                        "type": "integer",
                        "description": "The price range of the restaurants (1 = cheap to 4 = very expensive)"
                    }
                },
                "required": ["location"]
            }
        }
    }
]

In [83]:
messages=[
        {"role": "user", "content": "I'm in Malappuram, can you find a restaurant for me?"}
    ]

In [84]:
import json
prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,  # tool_list is a list of function dicts
    add_generation_prompt=True,
    tokenize=False
)

In [87]:
print(prompt)

<|begin_of_text|><start_of_turn>human
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.Here are the available tools:<tools> [{'type': 'function', 'function': {'name': 'search_restaurants', 'description': 'Search for restaurants in a specific location', 'parameters': {'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'The location to search for restaurants'}, 'cuisine': {'type': 'string', 'description': 'The cuisine type to filter the restaurants'}, 'price_range': {'type': 'integer', 'description': 'The price range of the restaurants (1 = cheap to 4 = very expensive)'}}, 'required': ['location']}}}]</tools>Use the following pydantic model json schema for each tool call you will make: {'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Argu

In [86]:
tokenizer.push_to_hub(peft_model_id)

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Abdulvajid/Llama_Qlora_Reasoning_ToolCalling_Finetuned_4Bit/commit/627f2d622bbeeedd8f45274070a0ef518ed5eb49', commit_message='Upload tokenizer', commit_description='', oid='627f2d622bbeeedd8f45274070a0ef518ed5eb49', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Abdulvajid/Llama_Qlora_Reasoning_ToolCalling_Finetuned_4Bit', endpoint='https://huggingface.co', repo_type='model', repo_id='Abdulvajid/Llama_Qlora_Reasoning_ToolCalling_Finetuned_4Bit'), pr_revision=None, pr_num=None)